In [36]:
import pandas as pd
from tqdm import tqdm
import requests
import os
import json

In [ ]:
df = pd.read_csv('Data Ekspektasi Grade FOOD - Sheet1.csv')
# Convert to key-value format with splitting
keyValueClown = {
    row['ITEM']: {
        "Nova Expectation": row['Nova Expectation'],
        "NutriScore Expectation": row['NutriScore Expectation'],
        "Actual Nova": [x.strip() for x in str(row['Actual Nova']).split(',') if pd.notna(x) and x.strip().lower() != 'nan'],
        "Actual NutriScore": [x.strip() for x in str(row['Actual NutriScore']).split(',') if pd.notna(x) and x.strip().lower() != 'nan']
    }
    for _, row in df.iterrows()
}

df.head()

,ITEM,Nova Expectation,NutriScore Expectation,Actual Nova,Actual NutriScore
0,Rice with Fried Chicken and Ketchup,3,D,4,D
1,Basmati Fried Rice with Fried Chicken and Sambal,3,C,3,C
2,"Bakso with Meat Stuffed Tofu, Fried Wonton Dum...",3,C,"4, 3, 3","D, C, C"
3,Black Crust Pizza with Beef Ham Italian Sausag...,3,D,4,D
4,Chocolate Ice cream drizzled with Chocolate sauce,4,D,4,D


In [42]:
json.dump(keyValueClown, open('keyValueClown.json', 'w'), indent=4)


ITEM	Nova Expectation	NutriScore Expectation	Actual Nova	Actual NutriScore
0	Rice with Fried Chicken and Ketchup	Three	D	Four	D
1	Basmati Fried Rice with Fried Chicken and Sambal	Three	C	Three	C
2	Bakso with Meat Stuffed Tofu, Fried Wonton Dum...	Three	C	Four, Three, Three	D, C, C
3	Black Crust Pizza with Beef Ham Italian Sausag...	Three	D	Four	D
4	Chocolate Ice cream drizzled with Chocolate sauce	Four	D	Four	D

In [32]:
mapping_dict_case_insensitive = {
    'FOUR' : 4,
    'THREE' : 3,
    'TWO' : 2,
    'ONE' : 1
}

In [ ]:
def upload_image_and_get_response(file_path):
    url = 'http://localhost:5128/Analysis/Method3'
    with open(file_path, 'rb') as f:
        files = {'image': (file_path, f, 'image/jpeg')}
        response = requests.post(url, files=files)
        
        # Check if the response contains JSON data
        if 'application/json' in response.headers.get('content-type', ''):
            return response.json()
        else:
            response.raise_for_status()



def update_dataframe_with_response(image_name, response_data):
    # Remove the extension from the image name
    image_name = os.path.splitext(image_name)[0]
    nova_classification = response_data['novaClassification']
    if str(nova_classification).upper() in mapping_dict_case_insensitive:
        nova_classification = mapping_dict_case_insensitive[str(nova_classification).upper()]

    if image_name not in keyValueClown:
        keyValueClown[image_name] = {
            "Actual Nova": [],
            "Actual NutriScore": []
        }

    keyValueClown[image_name]['Actual Nova'].append(nova_classification)
    keyValueClown[image_name]['Actual NutriScore'].append(response_data['nutriGrade'])
    


def upload_images_and_update_dataframe(directory, df):
    image_files = [f for f in os.listdir(directory) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    
    for filename in tqdm(image_files, desc="Uploading Images"):
        file_path = os.path.join(directory, filename)
        response_data = upload_image_and_get_response(file_path)
        update_dataframe_with_response(filename, response_data)
        print(f"Uploaded {filename} and updated DataFrame.")
        # Export the data to a new CSV file
        df = pd.DataFrame.from_dict(keyValueClown, orient='index').reset_index()
        df.to_csv('Updated.csv', index=False)
       

# Example usage
upload_images_and_update_dataframe('images', df)

Uploading Images:   3%|▎         | 1/37 [00:21<13:02, 21.73s/it]

Uploaded Spaghetti with sliced meatballs.jpeg and updated DataFrame.


Uploading Images:   5%|▌         | 2/37 [00:55<16:39, 28.54s/it]

Uploaded Rice with Smashed Fried Chicken and Sambal.jpg and updated DataFrame.


Uploading Images:   8%|▊         | 3/37 [01:21<15:37, 27.59s/it]

...

Uploaded Fried Rice with Omelette and sliced cucumber.jpg and updated DataFrame.


Uploading Images:  62%|██████▏   | 23/37 [10:08<05:55, 25.39s/it]

Uploaded Fried Rice with Meatballs, sliced carrots, sliced cucumber, and shrimp chips package.jpg and updated DataFrame.


Uploading Images:  65%|██████▍   | 24/37 [10:32<05:24, 24.97s/it]

Uploaded Fried Rice with Fried Egg and Sliced Cucumbers.jpg and updated DataFrame.


In [13]:
# Function to calculate accuracy for NutriScore
def calculate_accuracy_for_nutriscore(expectation, actual):
    # Convert actual to a list if it's a string with commas
    if isinstance(actual, str) and ',' in actual:
        actual = actual.split(', ')
    else:
        actual = [actual]
    
    # Mapping NutriScore letters to numbers
    score_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
    
    scores = []
    
    for act in actual:
        if expectation == act:
            scores.append(1.0)
        else:
            if expectation not in score_mapping or act not in score_mapping:
                continue
            exp_num = score_mapping[expectation]
            act_num = score_mapping.get(act, 0)  # Default to 0 if not found
            absolute_difference = abs(exp_num - act_num)
            scores.append(1 - (absolute_difference / 4))
    if len(scores) == 0:
        return None
    return sum(scores) / len(scores)

# Apply the function to calculate NutriScore accuracy
df['NutriScore Accuracy'] = df.apply(lambda row: calculate_accuracy_for_nutriscore(row['NutriScore Expectation'], row['Actual NutriScore']), axis=1)
df.head()

,ITEM,Nova Expectation,NutriScore Expectation,Actual Nova,Actual NutriScore,NutriScore Accuracy
0,Rice with Fried Chicken and Ketchup,3,D,4,D,1.000000
1,Basmati Fried Rice with Fried Chicken and Sambal,3,C,3,C,1.000000
2,"Bakso with Meat Stuffed Tofu, Fried Wonton Dum...",3,C,"4, 3, 3","D, C, C",0.916667
3,Black Crust Pizza with Beef Ham Italian Sausag...,3,D,4,D,1.000000
4,Chocolate Ice cream drizzled with Chocolate sauce,4,D,4,D,1.000000


In [14]:
# Function to calculate accuracy
def calculate_accuracy_for_nova(expectation, actual):
    if isinstance(actual, str) and ',' in actual:
        actual = actual.split(', ')
    else:
        actual = [actual]
    
    
    
    scores = []
    
    for act in actual:

        if act == 'nan' or act != act:
            return None
        if expectation == act:
            scores.append(1.0)
        else:
            act = int(act)
            expectation = int(expectation)
            absolute_difference = abs(act - expectation)
            scores.append(1 - (absolute_difference / 3))
    
    return sum(scores) / len(scores) 

# Apply the function to calculate accuracy
df['NOVA Accuracy'] = df.apply(lambda row: calculate_accuracy_for_nova(row['Nova Expectation'], row['Actual Nova']), axis=1)
df.head()

,ITEM,Nova Expectation,NutriScore Expectation,Actual Nova,Actual NutriScore,NutriScore Accuracy,NOVA Accuracy
0,Rice with Fried Chicken and Ketchup,3,D,4,D,1.000000,0.666667
1,Basmati Fried Rice with Fried Chicken and Sambal,3,C,3,C,1.000000,1.000000
2,"Bakso with Meat Stuffed Tofu, Fried Wonton Dum...",3,C,"4, 3, 3","D, C, C",0.916667,0.888889
3,Black Crust Pizza with Beef Ham Italian Sausag...,3,D,4,D,1.000000,0.666667
4,Chocolate Ice cream drizzled with Chocolate sauce,4,D,4,D,1.000000,1.000000


In [18]:
# Add row total accuracy for NutriScore
# filter Null values
nutri_score_accuracy_non_null_len = len(df[df['NutriScore Accuracy'].notnull()])
nutri_score_accuracy_sum = df['NutriScore Accuracy'].sum()
df.loc['Total', 'NutriScore Accuracy'] = nutri_score_accuracy_sum / nutri_score_accuracy_non_null_len

# Add row total accuracy for NOVA
# filter Null values
nova_accuracy_non_null_len = len(df[df['NOVA Accuracy'].notnull()])
nova_accuracy_sum = df['NOVA Accuracy'].sum()
df.loc['Total', 'NOVA Accuracy'] = nova_accuracy_sum / nova_accuracy_non_null_len 

df.head()

,ITEM,Nova Expectation,NutriScore Expectation,Actual Nova,Actual NutriScore,NutriScore Accuracy,NOVA Accuracy
0,Rice with Fried Chicken and Ketchup,3.0,D,4,D,1.000000,0.666667
1,Basmati Fried Rice with Fried Chicken and Sambal,3.0,C,3,C,1.000000,1.000000
2,"Bakso with Meat Stuffed Tofu, Fried Wonton Dum...",3.0,C,"4, 3, 3","D, C, C",0.916667,0.888889
3,Black Crust Pizza with Beef Ham Italian Sausag...,3.0,D,4,D,1.000000,0.666667
4,Chocolate Ice cream drizzled with Chocolate sauce,4.0,D,4,D,1.000000,1.000000


In [19]:
# Export the data to a new CSV file
df.to_csv('Computed.csv', index=False)